In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import time
import math
from mpl_toolkits.mplot3d import Axes3D  # Potrzebne do wykresów 3D

# Definicje algorytmów
def dominates(p1, p2, criteria):
    for k in range(len(p1)):
        if criteria[k] == 'min':
            if p1[k] > p2[k]:
                return False
        else:  # 'max'
            if p1[k] < p2[k]:
                return False
    return any(
        (p1[k] < p2[k] if criteria[k] == 'min' else p1[k] > p2[k]) for k in range(len(p1))
    )

def naiveNonFilter(X, criteria):
    P = []
    number_of_comparisons = 0
    X_temp = X.copy()

    while X_temp:
        Y = X_temp[0]
        dominated_by_Y = []
        dominates_Y = False
        for j in range(1, len(X_temp)):
            number_of_comparisons += 2
            Xj = X_temp[j]
            Y_dominates_Xj = all(
                Y[k] <= Xj[k] if criteria[k] == 'min' else Y[k] >= Xj[k]
                for k in range(len(Y))
            )
            Xj_dominates_Y = all(
                Xj[k] <= Y[k] if criteria[k] == 'min' else Xj[k] >= Y[k]
                for k in range(len(Y))
            )
            if Y_dominates_Xj:
                dominated_by_Y.append(Xj)
            elif Xj_dominates_Y:
                dominates_Y = True
                break  # Y jest zdominowany przez Xj
        if not dominates_Y:
            if Y not in P:
                P.append(Y)
            X_temp.remove(Y)
            for x in dominated_by_Y:
                if x in X_temp:
                    X_temp.remove(x)
        else:
            X_temp.remove(Y)
    return P, number_of_comparisons

def naiveWithFilter(X, criteria):
    P = []
    number_of_comparisons = 0
    X_temp = X.copy()

    while X_temp:
        Y = X_temp[0]
        dominated_by_Y = []
        dominates_Y = False
        for x_j in X_temp[1:]:
            number_of_comparisons += 2
            Y_dominates_Xj = all(
                Y[k] <= x_j[k] if criteria[k] == 'min' else Y[k] >= x_j[k]
                for k in range(len(Y))
            )
            Xj_dominates_Y = all(
                x_j[k] <= Y[k] if criteria[k] == 'min' else x_j[k] >= Y[k]
                for k in range(len(Y))
            )
            if Y_dominates_Xj:
                dominated_by_Y.append(x_j)
            elif Xj_dominates_Y:
                dominates_Y = True
                break  # Restartujemy z nowym Y
        if dominates_Y:
            X_temp.remove(Y)
            continue  # Restartujemy pętlę z nowym Y
        else:
            if Y not in P:
                P.append(Y)
            X_temp.remove(Y)
            for x in dominated_by_Y:
                if x in X_temp:
                    X_temp.remove(x)
    return P, number_of_comparisons

def distance(p1, p2):
    return math.sqrt(sum((p1[k] - p2[k]) ** 2 for k in range(len(p1))))

def calculate_ideal_point(X, criteria):
    ideal = []
    for k in range(len(X[0])):
        if criteria[k] == 'min':
            ideal.append(min(point[k] for point in X))
        else:
            ideal.append(max(point[k] for point in X))
    return tuple(ideal)

def naiveIdealPoint(X, criteria):
    ideal = calculate_ideal_point(X, criteria)

    distances = [(distance(p, ideal), idx) for idx, p in enumerate(X)]
    distances.sort(key=lambda x: x[0])

    sorted_indices = [index for _, index in distances]

    P = []
    number_of_comparisons = 0
    X_temp = X.copy()
    m = 0

    while X_temp and m < len(sorted_indices):
        current_p = X[sorted_indices[m]]
        if current_p in X_temp:
            P.append(current_p)
            X_temp.remove(current_p)

            dominated_p = []
            for p in X_temp:
                number_of_comparisons += 1
                if dominates(current_p, p, criteria):
                    dominated_p.append(p)

            for dp in dominated_p:
                if dp in X_temp:
                    X_temp.remove(dp)

        m += 1

    return P, number_of_comparisons

# Definicja aplikacji
class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Znajdowanie Punktów Niezdominowanych")
        self.geometry("1200x800")
        self.create_widgets()

    def create_widgets(self):
        # Konfiguracja danych wejściowych
        input_frame = ttk.LabelFrame(self, text="Konfiguracja danych wejściowych")
        input_frame.pack(fill="x", padx=10, pady=5)

        # Liczba punktów
        ttk.Label(input_frame, text="Liczba punktów:").grid(row=0, column=0, padx=5, pady=5)
        self.num_points_var = tk.IntVar(value=50)
        ttk.Entry(input_frame, textvariable=self.num_points_var).grid(row=0, column=1, padx=5, pady=5)

        # Liczba kryteriów
        ttk.Label(input_frame, text="Liczba kryteriów (2 lub 3):").grid(row=1, column=0, padx=5, pady=5)
        self.num_criteria_var = tk.IntVar(value=2)
        ttk.OptionMenu(input_frame, self.num_criteria_var, 2, 2, 3, command=self.update_criteria_inputs).grid(row=1, column=1, padx=5, pady=5)

        # Dynamiczne tworzenie pól dla kryteriów
        self.criteria_frames = []
        self.criteria_ranges = []
        self.criteria_minmax = []
        self.update_criteria_inputs()

        # Przyciski
        ttk.Button(input_frame, text="Generuj dane", command=self.generate_data).grid(row=5, column=0, padx=5, pady=5)
        ttk.Button(input_frame, text="Uruchom algorytmy", command=self.run_algorithms).grid(row=5, column=1, padx=5, pady=5)

        # Wybór algorytmów
        algo_frame = ttk.LabelFrame(self, text="Algorytmy")
        algo_frame.pack(fill="x", padx=10, pady=5)

        self.algo_vars = {}
        self.algo_names = ["Naive Non-Filter", "Naive With Filter", "Naive Ideal Point"]
        for i, name in enumerate(self.algo_names):
            var = tk.BooleanVar(value=True)
            self.algo_vars[name] = var
            ttk.Checkbutton(algo_frame, text=name, variable=var).grid(row=0, column=i, padx=5, pady=5)

        # Ramka wyników
        result_frame = ttk.Frame(self)
        result_frame.pack(fill="both", expand=True, padx=10, pady=5)

        self.result_text = tk.Text(result_frame, height=10)
        self.result_text.pack(fill="x", padx=5, pady=5)

        # Wykres
        self.figure = plt.Figure(figsize=(6, 5))
        self.canvas = FigureCanvasTkAgg(self.figure, master=result_frame)
        self.canvas.get_tk_widget().pack(fill="both", expand=True)

    def update_criteria_inputs(self, *args):
        # Usuwanie poprzednich pól
        for frame in self.criteria_frames:
            frame.destroy()
        self.criteria_frames.clear()
        self.criteria_ranges.clear()
        self.criteria_minmax.clear()

        num_criteria = self.num_criteria_var.get()
        input_frame = self.children['!labelframe']

        for i in range(num_criteria):
            frame = ttk.Frame(input_frame)
            frame.grid(row=2+i, column=0, columnspan=2, padx=5, pady=5, sticky="w")
            self.criteria_frames.append(frame)

            ttk.Label(frame, text=f"Kryterium {i+1} zakres (min, max):").grid(row=0, column=0, padx=5, pady=5)
            min_var = tk.DoubleVar(value=0)
            max_var = tk.DoubleVar(value=10)
            ttk.Entry(frame, textvariable=min_var, width=10).grid(row=0, column=1, padx=5, pady=5)
            ttk.Entry(frame, textvariable=max_var, width=10).grid(row=0, column=2, padx=5, pady=5)
            self.criteria_ranges.append((min_var, max_var))

            ttk.Label(frame, text=f"Kryterium {i+1} (Min/Max):").grid(row=0, column=3, padx=5, pady=5)
            minmax_var = tk.StringVar(value="Minimize")
            ttk.OptionMenu(frame, minmax_var, "Minimize", "Minimize", "Maximize").grid(row=0, column=4, padx=5, pady=5)
            self.criteria_minmax.append(minmax_var)

    def generate_data(self):
        num_points = self.num_points_var.get()
        num_criteria = self.num_criteria_var.get()

        self.X = []
        for _ in range(num_points):
            point = []
            for i in range(num_criteria):
                min_val = self.criteria_ranges[i][0].get()
                max_val = self.criteria_ranges[i][1].get()
                val = np.random.uniform(min_val, max_val)
                point.append(val)
            self.X.append(tuple(point))

        self.result_text.insert(tk.END, f"Wygenerowano {num_points} punktów z {num_criteria} kryteriami.\n")
        self.plot_data()

    def run_algorithms(self):
        selected_algos = [name for name in self.algo_names if self.algo_vars[name].get()]
        if not selected_algos:
            messagebox.showwarning("Brak wybranego algorytmu", "Proszę wybrać co najmniej jeden algorytm.")
            return

        if not hasattr(self, 'X') or not self.X:
            messagebox.showwarning("Brak danych", "Proszę najpierw wygenerować dane.")
            return

        # Pobierz kryteria
        criteria = []
        for minmax_var in self.criteria_minmax:
            if minmax_var.get() == "Minimize":
                criteria.append('min')
            else:
                criteria.append('max')

        # Wyczyszczenie poprzednich wyników
        self.result_text.delete(1.0, tk.END)
        self.figure.clear()  # Przeniesione poza pętlę

        colors = {'Naive Non-Filter': 'red', 'Naive With Filter': 'blue', 'Naive Ideal Point': 'green'}
        markers = {'Naive Non-Filter': 'o', 'Naive With Filter': '^', 'Naive Ideal Point': 's'}

        # Ustawienie osi wykresu
        if len(self.X[0]) == 2:
            self.ax = self.figure.add_subplot(111)
        elif len(self.X[0]) == 3:
            self.ax = self.figure.add_subplot(111, projection='3d')
        else:
            messagebox.showwarning("Wizualizacja", "Wizualizacja dostępna tylko dla 2 lub 3 kryteriów.")
            return

        # Wykres wszystkich punktów (przed pętlą)
        if len(self.X[0]) == 2:
            all_xs = [p[0] for p in self.X]
            all_ys = [p[1] for p in self.X]
            self.ax.scatter(all_xs, all_ys, color='gray', alpha=0.5, label='Wszystkie punkty')
            self.ax.set_xlabel('Kryterium 1')
            self.ax.set_ylabel('Kryterium 2')
        elif len(self.X[0]) == 3:
            all_xs = [p[0] for p in self.X]
            all_ys = [p[1] for p in self.X]
            all_zs = [p[2] for p in self.X]
            self.ax.scatter(all_xs, all_ys, all_zs, color='gray', alpha=0.5, label='Wszystkie punkty')
            self.ax.set_xlabel('Kryterium 1')
            self.ax.set_ylabel('Kryterium 2')
            self.ax.set_zlabel('Kryterium 3')

        for algo_name in selected_algos:
            start_time = time.time()
            if algo_name == "Naive Non-Filter":
                P, no = naiveNonFilter(self.X.copy(), criteria)
            elif algo_name == "Naive With Filter":
                P, no = naiveWithFilter(self.X.copy(), criteria)
            elif algo_name == "Naive Ideal Point":
                P, no = naiveIdealPoint(self.X.copy(), criteria)
            else:
                continue
            end_time = time.time()
            elapsed_time = end_time - start_time

            self.result_text.insert(tk.END, f"{algo_name}:\n")
            self.result_text.insert(tk.END, f"Liczba porównań: {no}\n")
            self.result_text.insert(tk.END, f"Czas wykonania: {elapsed_time:.6f} sekund\n")
            self.result_text.insert(tk.END, f"Liczba punktów niezdominowanych: {len(P)}\n\n")

            # Wykres punktów niezdominowanych
            if len(self.X[0]) == 2:
                xs = [p[0] for p in P]
                ys = [p[1] for p in P]
                self.ax.scatter(xs, ys, color=colors.get(algo_name, 'black'), marker=markers.get(algo_name, 'o'), label=f"{algo_name}")
            elif len(self.X[0]) == 3:
                xs = [p[0] for p in P]
                ys = [p[1] for p in P]
                zs = [p[2] for p in P]
                self.ax.scatter(xs, ys, zs, color=colors.get(algo_name, 'black'), marker=markers.get(algo_name, 'o'), label=f"{algo_name}")

        self.ax.legend()
        self.canvas.draw()

    def plot_data(self):
        self.figure.clear()
        if len(self.X[0]) == 2:
            self.ax = self.figure.add_subplot(111)
            xs = [p[0] for p in self.X]
            ys = [p[1] for p in self.X]
            self.ax.scatter(xs, ys, color='gray', alpha=0.5, label='Wszystkie punkty')
            self.ax.set_xlabel('Kryterium 1')
            self.ax.set_ylabel('Kryterium 2')
        elif len(self.X[0]) == 3:
            self.ax = self.figure.add_subplot(111, projection='3d')
            xs = [p[0] for p in self.X]
            ys = [p[1] for p in self.X]
            zs = [p[2] for p in self.X]
            self.ax.scatter(xs, ys, zs, color='gray', alpha=0.5, label='Wszystkie punkty')
            self.ax.set_xlabel('Kryterium 1')
            self.ax.set_ylabel('Kryterium 2')
            self.ax.set_zlabel('Kryterium 3')
        else:
            messagebox.showwarning("Wizualizacja", "Wizualizacja dostępna tylko dla 2 lub 3 kryteriów.")
            return
        self.canvas.draw()

if __name__ == "__main__":
    app = App()
    app.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\Python39\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\lszys\AppData\Local\Temp\ipykernel_7288\4186207901.py", line 294, in run_algorithms
    P, no = naiveNonFilter(self.X.copy(), criteria)
  File "C:\Users\lszys\AppData\Local\Temp\ipykernel_7288\4186207901.py", line 39, in naiveNonFilter
    Xj_dominates_Y = all(
  File "C:\Users\lszys\AppData\Local\Temp\ipykernel_7288\4186207901.py", line 40, in <genexpr>
    Xj[k] <= Y[k] if criteria[k] == 'min' else Xj[k] >= Y[k]
IndexError: list index out of range
